In [ ]:
# pip install openai==0.28
import openai
# from openai import OpenAI
import os
import logging
from langchain.prompts import PromptTemplate
import pandas as pd
from dotenv import load_dotenv
from flask import Flask, request, jsonify

In [2]:
load_dotenv()
azure_openai_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
# Set up logging configuration
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Configure OpenAI with Azure settings
openai.api_type = "azure"
openai.api_key = azure_openai_key
openai.api_base = azure_openai_endpoint
openai.api_version = "2023-05-15"
# client = OpenAI(api_key=azure_openai_key)

In [33]:
# # Load inventory data (example CSV file)
# inventory_data = pd.DataFrame({
#     "product_id": [1, 2, 3],
#     "product_name": ["Widget A", "Widget B", "Widget C"],
#     "stock_level": [500, 200, 50],
#     "sales_last_month": [50, 150, 45]
# })

# # Define overstock threshold
# OVERSTOCK_THRESHOLD = 300

# # Initialize OpenAI GPT model
# llm = openai(model="gpt-4", temperature=0.7)
# llm = openai(
#         engine=azure_openai_deployment,
#         messages=messages,
#         temperature=0.7,
#         max_tokens=800
#     )

# # Define a function to detect overstock
# def detect_overstock(data):
#     overstocked_items = data[data["stock_level"] > OVERSTOCK_THRESHOLD]
#     return overstocked_items

# # Define a function to generate promotional discounts
# def generate_promotions(overstocked_items):
#     promotions = []
#     for _, row in overstocked_items.iterrows():
#         prompt = PromptTemplate(
#             input_variables=["product_name", "stock_level"],
#             template=(
#                 "The product '{product_name}' has a stock level of {stock_level}, "
#                 "which is above the threshold. Suggest a promotional discount strategy."
#             )
#         )
#         response = llm(prompt.format(product_name=row["product_name"], stock_level=row["stock_level"]))
#         promotions.append({"product_id": row["product_id"], "promotion": response})
#     return promotions

# # Detect overstocked items
# overstocked_items = detect_overstock(inventory_data)

# # Generate promotions for overstocked items
# if not overstocked_items.empty:
#     promotions = generate_promotions(overstocked_items)
#     for promo in promotions:
#         print(f"Product ID: {promo['product_id']}, Promotion: {promo['promotion']}")
# else:
#     print("No overstock detected.")

# Price Optimization Agent
This section demonstrates how to use the Azure OpenAI deployed model to create a price optimization agent. The agent analyzes product data and suggests optimal pricing strategies based on stock levels, sales trends, and other factors.

In [3]:
# Define a function for price optimization
def optimize_price(product_name, stock_level, sales_last_month):
    try:        
        prompt = PromptTemplate(
            input_variables=['product_name', 'stock_level', 'sales_last_month'],
            template=(
                "The product '{product_name}' has a stock level of {stock_level} and sales last month were {sales_last_month}. "
                "Suggest an optimal pricing strategy to maximize revenue and reduce overstock."
            )
        )
        response = openai.ChatCompletion.create(
                        engine=azure_openai_deployment,
                        messages=[{"role": "system", "content": "You are a pricing optimization assistant."},
                                  {"role": "user", "content": prompt.format(product_name=product_name, stock_level=stock_level, sales_last_month=sales_last_month)}
            ],
            temperature=0.7,
            max_tokens=800
        )
        ai_message = response['choices'][0]['message']['content']
        return ai_message
    except Exception as e:
        logger.error(f"Error in analyze-chat api: {str(e)}")        
        # Return a JSON response with the error message
        return str(e)

In [4]:
import pandas as pd

# Load the CSV data
data_path = 'price_optimization_data.csv'
data = pd.read_csv(data_path)

# Iterate through the data and optimize prices
optimized_results = []
for index, row in data.iterrows():
    product_name = row['Product Name']
    stock_level = row['Stock Level']
    sales_last_month = row['Sales Last Month']
    optimal_price_strategy = optimize_price(product_name, stock_level, sales_last_month)
    optimized_results.append({
        'Product Name': product_name,
        'Optimal Strategy': optimal_price_strategy
    })

# Convert results to a DataFrame
optimized_df = pd.DataFrame(optimized_results)

# Save the results to a new CSV file
optimized_df.to_csv('optimized_price_strategies.csv', index=False)

print('Price optimization completed. Results saved to ai-agents/optimized_price_strategies.csv')

2025-04-28 11:41:07,972 - INFO - error_code=403 error_message='Access denied due to Virtual Network/Firewall rules.' error_param=None error_type=None message='OpenAI API error received' stream_error=False
2025-04-28 11:41:07,972 - ERROR - Error in analyze-chat api: Access denied due to Virtual Network/Firewall rules.
2025-04-28 11:41:07,972 - ERROR - Error in analyze-chat api: Access denied due to Virtual Network/Firewall rules.
2025-04-28 11:41:08,240 - INFO - error_code=403 error_message='Access denied due to Virtual Network/Firewall rules.' error_param=None error_type=None message='OpenAI API error received' stream_error=False
2025-04-28 11:41:08,243 - ERROR - Error in analyze-chat api: Access denied due to Virtual Network/Firewall rules.
2025-04-28 11:41:08,240 - INFO - error_code=403 error_message='Access denied due to Virtual Network/Firewall rules.' error_param=None error_type=None message='OpenAI API error received' stream_error=False
2025-04-28 11:41:08,243 - ERROR - Error in 

Price optimization completed. Results saved to ai-agents/optimized_price_strategies.csv


In [9]:
os.getcwd()

'e:\\projects\\capgemini-ai-agent\\ai-agents'